In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2  trl==0.4.7

In [ ]:
!pip install -U transformers trl

In [ ]:
!pip install -q accelerate peft==0.4.0 bitsandbytes==0.40.2  trl

In [ ]:
!pip install -U accelerate peft bitsandbytes

In [ ]:
!pip install -q accelerate peft

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,AutoModel,
    TrainingArguments,
    pipeline,
    Trainer
)
from peft import LoraConfig, PeftModel,get_peft_config,get_peft_model
from trl import SFTTrainer

2024-06-20 17:43:51.282371: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 17:43:51.282439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 17:43:51.284099: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
# The model that you want to train from the Hugging Face hub
model_name ="llava-hf/llava-1.5-7b-hf"

# The instruction dataset to use

# Fine-tuned model name
new_model = "llava-hf/llava-1.5-7b-hf"
# LoRA attention dimension
lora_r = 16
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.05
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

use_nested_quant = False

output_dir = "./results"

# Number of training epochs
num_train_epochs = 1
fp16 = False
bf16 = False

per_device_train_batch_size =1

per_device_eval_batch_size = 4

gradient_accumulation_steps = 8

gradient_checkpointing = True

max_grad_norm = 0.3

learning_rate = 3e-5

weight_decay = 0.001

optim = "paged_adamw_32bit"

lr_scheduler_type = "constant"

max_steps = -1
warmup_ratio = 0.03
group_by_length = True

save_steps = 100

logging_steps = 25

max_seq_length = False
packing = False
#device_map = {"": 0}

In [8]:
from transformers import AutoProcessor, AutoModelForPreTraining,LlavaForConditionalGeneration

In [9]:
# Load dataset (you can process it here)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = LlavaForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=bnb_config
)
#model.config.use_cache = False
#model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoProcessor.from_pretrained(model_name)
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj'],
    bias="none",
    task_type="CAUSAL_LM",
)

config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/llava/configuration_llava.py:100: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
import pandas as pd
data = pd.read_csv("/kaggle/input/false-true-images-for-minicpm/make-true-false-image-db/data.csv")

In [38]:
tokenizer.padding_side = "right"

In [39]:
data=data.sample(len(data))

In [13]:
from PIL import Image

In [56]:
#Datasets and Dataloaders
from torch.utils.data import Dataset, DataLoader

class QADataset(Dataset):
    def __init__(self, encodings):
        self.inputs = encodings
        
    def __getitem__(self, idx):
        d=self.inputs.iloc[idx]
        d=self.inputs.iloc[idx]
        text="USER: <image>\nanswer with true or false analysis the two images and determine if they the same or not? ASSISTANT:"
        labels=d['labels']
        path="/kaggle/input/false-true-images-for-minicpm/make-true-false-image-db/train/"+d['merged']
        image=Image.open(path).resize((448,448))
        with torch.set_grad_enabled(True):
                preproces = tokenizer(text=text+str(d[-1]), images=image, return_tensors="pt",truncation=True,)
                for i in  preproces.keys():
                   preproces[i]=preproces[i].squeeze(0).cuda()


                  #  preproces[i].requie_gradient=True
                return preproces
    def __len__(self):
        return len(self.inputs)
train_dataset = QADataset(data.iloc[0:1400].reset_index(drop=True))
val_dataset = QADataset(data.iloc[-100:].reset_index(drop=True))


In [ ]:
data[data['labels']==False]

In [14]:
from PIL import Image
import json

In [ ]:
combine=[]
for i in os.listdir("/kaggle/input/false-true-images-for-minicpm/make-true-false-image-db/train/train_json"):
    with open("/kaggle/input/false-true-images-for-minicpm/make-true-false-image-db/train/train_json/"+i,"r") as f:
        combine.append(json.load(f))

In [ ]:
!pip install deepspeed

In [ ]:
import glob
import json
import logging
import os
from dataclasses import dataclass, field
from functools import partial
from typing import Dict, List, Optional, Union, Literal, Tuple
from types import MethodType
import torch
import transformers
from accelerate.utils import DistributedType
from deepspeed import zero
from deepspeed.runtime.zero.partition_parameters import ZeroParamStatus

from transformers import AutoModel, AutoTokenizer
from transformers.integrations import deepspeed
from transformers import AutoModel, AutoTokenizer

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:
compine[0]

In [ ]:
model.eval()

In [49]:
"""out=0
o=1
for index,values in data.iterrows(): 
        text="USER: <image> \nanswer with true or false,analysis the two images and determine if they the same or not? ASSISTANT:"
        image=Image.open("/kaggle/input/false-true-images-for-minicpm/make-true-false-image-db/train/"+values['merged']).resize((512,512))

        #preproces = tokenizer(text=text, images=image ,return_tensors="pt")
        with torch.inference_mode():
            inputs=tokenizer(text=text, images=image, return_tensors="pt")
            generate_ids = model.generate(**inputs, max_new_tokens=15)
            print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0],values['labels'])
print(out/len(data))
image"""

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


USER:   
answer with true or false,analysis the two images and determine if they the same or not? ASSISTANT: The two images are not the same. The first image shows a black and False
USER:   
answer with true or false,analysis the two images and determine if they the same or not? ASSISTANT: The two images are not the same. The first image shows a triangle with False
USER:   
answer with true or false,analysis the two images and determine if they the same or not? ASSISTANT: The two images are not the same. The first image is a diagram of True
USER:   
answer with true or false,analysis the two images and determine if they the same or not? ASSISTANT: The two images are not the same. The first image is a graph with True
USER:   
answer with true or false,analysis the two images and determine if they the same or not? ASSISTANT: The two images are not the same. The first image features a square with True
USER:   
answer with true or false,analysis the two images and determine if they the sam


KeyboardInterrupt



In [50]:

!pip install wandb
import wandb
wandb.login(key="14459c516497ab76a78f7fc1278bfe60d301d250")

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#peftmodel.enable_input_require_grads()

In [51]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim=optim,
    save_steps=40,
    logging_steps=15,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16, 
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=False,
    lr_scheduler_type=lr_scheduler_type,
   gradient_checkpointing=gradient_checkpointing
)


In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim=optim,
    save_steps=40,
    logging_steps=15,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16, 
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=False,
    lr_scheduler_type=lr_scheduler_type,
   gradient_checkpointing=gradient_checkpointing
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
      max_seq_length=18,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    packing=False
)
trainer.train()
trainer.model.save_pretrained(new_model)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer

Step,Training Loss


In [ ]:
trainer.evaluate()


In [ ]:
! pip install evaluate